In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import pandas_datareader as pdr
import itertools
from sklearn.preprocessing import StandardScaler
%matplotlib inline

In [2]:
aal = pdr.get_data_google('AAL', '2000-01-01', '2017-01-01')

In [3]:
aal.drop('Volume', axis=1, inplace=True)
aal.head()

,Open,High,Low,Close
Date,,,,
2013-12-09,23.85,25.44,23.45,24.60
2013-12-10,24.50,25.17,24.41,24.88
2013-12-11,25.48,27.20,25.37,25.99
2013-12-12,26.20,26.71,25.45,25.45
2013-12-13,25.75,26.30,25.52,26.23


In [4]:
def _set_index_column_if_necessary(data: pd.DataFrame) -> pd.DataFrame:
    if 'Date' in data.columns:
        data.set_index('Date', inplace=True)
    return data

In [5]:
aal = _set_index_column_if_necessary(aal)
cols = aal.columns
aal.head()

,Open,High,Low,Close
Date,,,,
2013-12-09,23.85,25.44,23.45,24.60
2013-12-10,24.50,25.17,24.41,24.88
2013-12-11,25.48,27.20,25.37,25.99
2013-12-12,26.20,26.71,25.45,25.45
2013-12-13,25.75,26.30,25.52,26.23


In [6]:
cols

Index(['Open', 'High', 'Low', 'Close'], dtype='object')

In [7]:
return_days = 2

In [8]:
def feature(data, first_col, second_col, base_col):
    return (data[first_col]-data[second_col])/data[base_col]

In [10]:
def create_extra_columns(data, cols):
    pool = []
    for left, right in itertools.product(cols, cols):
        pair1 = left + right
        pair2 = right + left
        if left != right and pair1 not in pool and pair2 not in pool:
            aal[left + '/' + right] = feature(data, left, right, 'Close')
            pool.append(pair1)

In [11]:
create_extra_columns(aal, cols)

In [12]:
aal.columns

Index(['Open', 'High', 'Low', 'Close', 'Open/High', 'Open/Low', 'Open/Close',
       'High/Low', 'High/Close', 'Low/Close'],
      dtype='object')

In [13]:
days=[5,10]

for col, day in itertools.product(cols, days):
    aal[col + ' ' + str(day) + ' MA'] = aal[col].rolling(day).mean()
    aal[col + ' ' + str(day) + ' max'] = aal[col].rolling(day).max()
    aal[col + ' ' + str(day) + ' min'] = aal[col].rolling(day).min()
aal.dropna(inplace=True)

In [14]:
aal.columns

Index(['Open', 'High', 'Low', 'Close', 'Open/High', 'Open/Low', 'Open/Close',
       'High/Low', 'High/Close', 'Low/Close', 'Open 5 MA', 'Open 5 max',
       'Open 5 min', 'Open 10 MA', 'Open 10 max', 'Open 10 min', 'High 5 MA',
       'High 5 max', 'High 5 min', 'High 10 MA', 'High 10 max', 'High 10 min',
       'Low 5 MA', 'Low 5 max', 'Low 5 min', 'Low 10 MA', 'Low 10 max',
       'Low 10 min', 'Close 5 MA', 'Close 5 max', 'Close 5 min', 'Close 10 MA',
       'Close 10 max', 'Close 10 min'],
      dtype='object')

In [15]:
rolling_features = list(filter(lambda col: (' MA' in col) or (' max' in col) or (' min' in col), aal.columns))
rolling_features

['Open 5 MA',
 'Open 5 max',
 'Open 5 min',
 'Open 10 MA',
 'Open 10 max',
 'Open 10 min',
 'High 5 MA',
 'High 5 max',
 'High 5 min',
 'High 10 MA',
 'High 10 max',
 'High 10 min',
 'Low 5 MA',
 'Low 5 max',
 'Low 5 min',
 'Low 10 MA',
 'Low 10 max',
 'Low 10 min',
 'Close 5 MA',
 'Close 5 max',
 'Close 5 min',
 'Close 10 MA',
 'Close 10 max',
 'Close 10 min']

In [16]:
create_extra_columns(aal, rolling_features)

In [17]:
ret = 100*aal['Close'].pct_change(return_days).shift(-return_days)

features_to_drop = list(filter(lambda col: '/' not in col, aal.columns))
aal.drop(features_to_drop, axis=1, inplace=True)

aal['Return'] = ret
aal.head(10)

,Open/High,Open/Low,Open/Close,High/Low,High/Close,Low/Close,Open 5 MA/Open 5 max,Open 5 MA/Open 5 min,Open 5 MA/Open 10 MA,Open 5 MA/Open 10 max,...,Close 5 max/Close 10 MA,Close 5 max/Close 10 max,Close 5 max/Close 10 min,Close 5 min/Close 10 MA,Close 5 min/Close 10 max,Close 5 min/Close 10 min,Close 10 MA/Close 10 max,Close 10 MA/Close 10 min,Close 10 max/Close 10 min,Return
Date,,,,,,,,,,,,,,,,,,,,,
2013-12-20,-0.011774,0.001519,-0.005697,0.013293,0.006077,-0.007216,-0.013293,0.011014,0.021344,-0.013293,...,0.028712,0.000000,0.076339,0.009343,-0.019370,0.056969,-0.028712,0.047626,0.076339,-0.303836
2013-12-23,-0.007639,0.009167,0.004202,0.016807,0.011841,-0.004966,-0.010237,0.008480,0.009549,-0.015966,...,0.012147,-0.010695,0.055386,0.003361,-0.019481,0.046600,-0.022842,0.043239,0.066081,-0.190985
2013-12-24,-0.009905,0.000000,-0.009524,0.009905,0.000381,-0.009524,-0.006629,0.004800,0.000152,-0.019581,...,0.006895,-0.010667,0.033524,-0.001105,-0.018667,0.025524,-0.017562,0.026629,0.044190,-4.990476
2013-12-26,-0.009185,0.005358,-0.000383,0.014543,0.008802,-0.005741,-0.005664,0.005434,-0.001301,-0.018676,...,0.006391,-0.010716,0.033678,-0.001646,-0.018752,0.025641,-0.017107,0.027287,0.044393,-5.166475
2013-12-27,-0.006014,0.041700,0.040497,0.047715,0.046512,-0.001203,-0.007298,0.006335,-0.001724,-0.020930,...,0.008741,-0.011227,0.055734,-0.046993,-0.066961,0.000000,-0.019968,0.046993,0.066961,1.242983
2013-12-30,-0.015335,0.008878,0.003632,0.024213,0.018967,-0.005246,-0.017918,0.039387,-0.008757,-0.031638,...,0.011421,-0.014528,0.059322,-0.047902,-0.073850,0.000000,-0.025948,0.047902,0.073850,2.340597
2013-12-31,-0.020198,0.004356,-0.020198,0.024554,0.000000,-0.024554,-0.023129,0.031525,-0.013386,-0.037386,...,0.016594,-0.003168,0.058218,-0.041624,-0.061386,0.000000,-0.019762,0.041624,0.061386,5.108911
2014-01-02,-0.029574,0.000394,-0.011435,0.029968,0.018139,-0.011830,-0.030363,0.024054,-0.015103,-0.037066,...,0.014708,-0.007886,0.053233,-0.038525,-0.061120,0.000000,-0.022595,0.038525,0.061120,6.585174
2014-01-03,-0.037679,0.009043,-0.029766,0.046722,0.007913,-0.038809,-0.025396,0.020196,-0.016315,-0.038206,...,0.028335,0.000000,0.066315,-0.037980,-0.066315,0.000000,-0.028335,0.037980,0.066315,1.356443


In [19]:
len(aal.columns)

283

In [20]:
list(aal.columns)

['Open/High',
 'Open/Low',
 'Open/Close',
 'High/Low',
 'High/Close',
 'Low/Close',
 'Open 5 MA/Open 5 max',
 'Open 5 MA/Open 5 min',
 'Open 5 MA/Open 10 MA',
 'Open 5 MA/Open 10 max',
 'Open 5 MA/Open 10 min',
 'Open 5 MA/High 5 MA',
 'Open 5 MA/High 5 max',
 'Open 5 MA/High 5 min',
 'Open 5 MA/High 10 MA',
 'Open 5 MA/High 10 max',
 'Open 5 MA/High 10 min',
 'Open 5 MA/Low 5 MA',
 'Open 5 MA/Low 5 max',
 'Open 5 MA/Low 5 min',
 'Open 5 MA/Low 10 MA',
 'Open 5 MA/Low 10 max',
 'Open 5 MA/Low 10 min',
 'Open 5 MA/Close 5 MA',
 'Open 5 MA/Close 5 max',
 'Open 5 MA/Close 5 min',
 'Open 5 MA/Close 10 MA',
 'Open 5 MA/Close 10 max',
 'Open 5 MA/Close 10 min',
 'Open 5 max/Open 5 min',
 'Open 5 max/Open 10 MA',
 'Open 5 max/Open 10 max',
 'Open 5 max/Open 10 min',
 'Open 5 max/High 5 MA',
 'Open 5 max/High 5 max',
 'Open 5 max/High 5 min',
 'Open 5 max/High 10 MA',
 'Open 5 max/High 10 max',
 'Open 5 max/High 10 min',
 'Open 5 max/Low 5 MA',
 'Open 5 max/Low 5 max',
 'Open 5 max/Low 5 min',

# X/y spit

In [21]:
test_date = datetime(2015,1,1).strftime('%Y-%m-%d')
aal.dropna(inplace=True)
X_data = aal.drop('Return', axis=1)
y_data = aal['Return']

In [23]:
X_data.head(2)

,Open/High,Open/Low,Open/Close,High/Low,High/Close,Low/Close,Open 5 MA/Open 5 max,Open 5 MA/Open 5 min,Open 5 MA/Open 10 MA,Open 5 MA/Open 10 max,...,Close 5 max/Close 5 min,Close 5 max/Close 10 MA,Close 5 max/Close 10 max,Close 5 max/Close 10 min,Close 5 min/Close 10 MA,Close 5 min/Close 10 max,Close 5 min/Close 10 min,Close 10 MA/Close 10 max,Close 10 MA/Close 10 min,Close 10 max/Close 10 min
Date,,,,,,,,,,,,,,,,,,,,,
2013-12-20,-0.011774,0.001519,-0.005697,0.013293,0.006077,-0.007216,-0.013293,0.011014,0.021344,-0.013293,...,0.019370,0.028712,0.000000,0.076339,0.009343,-0.019370,0.056969,-0.028712,0.047626,0.076339
2013-12-23,-0.007639,0.009167,0.004202,0.016807,0.011841,-0.004966,-0.010237,0.008480,0.009549,-0.015966,...,0.008785,0.012147,-0.010695,0.055386,0.003361,-0.019481,0.046600,-0.022842,0.043239,0.066081


In [24]:
y_data.head(2)

Date
2013-12-20   -0.303836
2013-12-23   -0.190985
Name: Return, dtype: float64

In [25]:
extreme = 3
def series_to_binarized_columns(y):
    pos = y > extreme
    neg = y < -extreme
    meds = (y > -extreme) & (y < extreme)
    y = np.array([neg, meds, pos]).T
    return y

In [26]:
X_train = X_data[:test_date]
X_test = X_data[test_date:]
y_train = y_data[:test_date]
y_test = y_data[test_date:]

In [27]:
y_train = series_to_binarized_columns(y_train)
y_test = series_to_binarized_columns(y_test)

In [28]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train) 
X_test = scaler.transform(X_test)

# KERAS

In [29]:
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, BatchNormalization
from keras.optimizers import Adam
from keras.models import load_model

Using TensorFlow backend.


In [30]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report

In [31]:
X_train.shape

(259, 282)

In [33]:
model = Sequential()
model.add(Dense(100, input_shape=(282,)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(50))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(3, kernel_initializer='uniform'))
model.add(Activation('softmax'))

model.compile(optimizer=Adam(),
              #loss=self.create_entropy(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=16, epochs=100, verbose=2)

Train on 259 samples, validate on 502 samples
Epoch 1/100
 - 2s - loss: 1.0730 - acc: 0.4402 - val_loss: 1.0164 - val_acc: 0.6155
Epoch 2/100
 - 0s - loss: 0.9618 - acc: 0.6293 - val_loss: 0.9589 - val_acc: 0.6793
Epoch 3/100
 - 0s - loss: 0.9073 - acc: 0.6216 - val_loss: 0.9200 - val_acc: 0.6793
Epoch 4/100
 - 0s - loss: 0.8689 - acc: 0.6332 - val_loss: 0.9253 - val_acc: 0.6693
Epoch 5/100
 - 0s - loss: 0.8629 - acc: 0.6293 - val_loss: 0.9147 - val_acc: 0.6753
Epoch 6/100
 - 0s - loss: 0.8361 - acc: 0.6448 - val_loss: 0.9151 - val_acc: 0.6594
Epoch 7/100
 - 0s - loss: 0.8341 - acc: 0.6371 - val_loss: 0.9146 - val_acc: 0.6773
Epoch 8/100
 - 0s - loss: 0.8112 - acc: 0.6332 - val_loss: 0.9087 - val_acc: 0.6554
Epoch 9/100
 - 0s - loss: 0.8073 - acc: 0.6255 - val_loss: 0.9062 - val_acc: 0.6673
Epoch 10/100
 - 0s - loss: 0.8225 - acc: 0.6216 - val_loss: 0.9120 - val_acc: 0.6514
Epoch 11/100
 - 0s - loss: 0.7862 - acc: 0.6448 - val_loss: 0.9199 - val_acc: 0.6554
Epoch 12/100
 - 0s - loss: 0